In [12]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.applications import Xception
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

In [17]:
print(tf.__version__)

2.17.0


In [13]:
# Load the two models
model1 = tf.keras.models.load_model('xception_train1.h5')
model2 = tf.keras.models.load_model('xception_train2.h5')

# Ensure that both models have the same architecture
assert model1.count_params() == model2.count_params(), "Models do not have the same architecture!"

# Extract weights from both models
weights1 = model1.get_weights()
weights2 = model2.get_weights()

In [14]:
print(f"Model 1 weights: {len(weights1)}")
print(f"Model 2 weights: {len(weights2)}")


Model 1 weights: 238
Model 2 weights: 238


In [15]:
# Ensure that both models have the same architecture
assert model1.count_params() == model2.count_params(), "Models do not have the same architecture!"

# Extract weights from both models
weights1 = model1.get_weights()
weights2 = model2.get_weights()

# Initialize a new model by copying the architecture and weights from model1
merged_model = tf.keras.models.clone_model(model1)
merged_model.set_weights(weights1)

# Update the weights with the average weights
average_weights = [(w1 + w2) / 2 for w1, w2 in zip(weights1, weights2)]
merged_model.set_weights(average_weights)

# Save the merged model
merged_model.save('merged_model.h5')

print("Merged model saved successfully.")


Merged model saved successfully.


In [16]:
# Load and preprocess the dataset for testing
test_datagen = ImageDataGenerator(rescale=1.0/255.0)

test_generator = test_datagen.flow_from_directory(
    r"pneumonia-detection\split_chest_xray\test",  # Replace with the path to your testing dataset directory
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

Found 624 images belonging to 2 classes.


In [23]:
model3 = tf.keras.models.load_model('merged_model.h5')
model3.compile(optimizer='adam',  # or your preferred optimizer
               loss='categorical_crossentropy',  # or your preferred loss function
               metrics=['accuracy'])  # or your preferred metrics


In [24]:
# Evaluate the model on the testing data
test_loss, test_accuracy = model3.evaluate(test_generator)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

c:\Users\Tejas\OneDrive\Desktop\T_work\1. Projects\A.I\CNN Project\myenv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


20/20 ━━━━━━━━━━━━━━━━━━━━ 36s 2s/step - accuracy: 0.6270 - loss: 0.5258
Test Loss: 0.5288372039794922, Test Accuracy: 0.625
